In [10]:
!pip install pandas
!pip install numpy
!pip install boto3
!pip install dspy-ai
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Using cached scikit_learn-1.5.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp312-cp312-macosx_12_0_arm64.whl (11.0 MB)
Using cached scipy-1.14.0-cp312-cp312-macosx_14_0_arm64.whl (23.1 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
df = pd.read_csv('./data/train.tsv', sep='\t')
print(len(df))

68


In [5]:
import dspy
from typing_extensions import Literal

class Classify(dspy.Signature):
    """Classify a text into one of the given options"""

    text = dspy.InputField()
        
    answer: Literal['Billing Inquiries','Policy Administration','Claims Assistance','Coverage Explanations','Quotes and Proposals','Account Management','Billing Disputes','Claims Disputes','Policy Comparisons','General Inquiries'] = dspy.OutputField(desc="Only one word. From the given list")

/Users/wckd/Desktop/dspy/DSPyClassification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from dspy.functional import TypedChainOfThought
class GetLabel(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_label = dspy.TypedChainOfThought(Classify)
    
    def forward(
            self,
            text
    ):
        label = self.generate_label(text = text)
        
        return label

In [18]:
# Create a dataset
dataset = []
for i, row in df.iterrows():
    example = dspy.Example(text = row['text'], answer=row['label'])
    dataset.append(example.with_inputs('text'))

In [19]:
import dspy
import random

# Does backoff by 3, 9, 27, 60 seconds.
def exponential_backoff(attempts):
    return min(3**attempts, 60)

bedrock_claude = dspy.AWSAnthropic(
    aws_provider = dspy.Bedrock(region_name="us-east-1", profile_name="stage"),
    model="anthropic.claude-3-5-sonnet-20240620-v1:0",
)
dspy.settings.configure(backoff_time=exponential_backoff)
dspy.settings.configure(lm=bedrock_claude)

In [21]:
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

classifier = GetLabel()

gpt4o_preds = []
true_labels = []
for data_point in tqdm(dataset, total=len(dataset)):
    gpt4o_preds.append(
        classifier(**data_point.inputs()).answer
    )
    true_labels.append(data_point.answer)

print("Classification report")
print(classification_report(true_labels, gpt4o_preds))
print("Confusion Matrix")
print(confusion_matrix(true_labels, gpt4o_preds))

100%|██████████| 68/68 [04:49<00:00,  4.26s/it]

Classification report
                       precision    recall  f1-score   support

   Account Management       1.00      0.83      0.91         6
     Billing Disputes       0.78      0.78      0.78         9
    Billing Inquiries       0.57      0.67      0.62         6
    Claims Assistance       0.86      1.00      0.92         6
      Claims Disputes       1.00      0.88      0.93         8
Coverage Explanations       0.40      1.00      0.57         8
    General Inquiries       0.00      0.00      0.00         9
Policy Administration       1.00      0.83      0.91         6
   Policy Comparisons       1.00      1.00      1.00         5
 Quotes and Proposals       1.00      0.60      0.75         5

             accuracy                           0.74        68
            macro avg       0.76      0.76      0.74        68
         weighted avg       0.72      0.74      0.70        68

Confusion Matrix
[[5 0 1 0 0 0 0 0 0 0]
 [0 7 2 0 0 0 0 0 0 0]
 [0 2 4 0 0 0 0 0 0 0]
 [0 0 0


/Users/wckd/Desktop/dspy/DSPyClassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/wckd/Desktop/dspy/DSPyClassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/wckd/Desktop/dspy/DSPyClassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w